In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from numba import jit

# 1a

In [4]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

def getAcc(pos : torch.Tensor, mass : torch.Tensor, G : float, softening : float):
    """
    Calculate the acceleration on each particle due to Newton's Law
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    # positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]
    
    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)**(-1.5)
    
    ax = G * (dx * inv_r3) @ mass
    ay = G * (dy * inv_r3) @ mass
    az = G * (dz * inv_r3) @ mass
    
    # pack together the acceleration components
    a = torch.hstack((ax,ay,az))

    return a

def leapfrog(pos: torch.Tensor, vec: torch.Tensor, mass: torch.Tensor, G: float, softening: float, dt: float):

    # leapfrog for N-body
    vec = vec + 0.5*dt*getAcc(pos, mass, G, softening)
    pos = pos+ vec*dt
    vec = vec+ 0.5*dt*getAcc(pos, mass, G, softening)

    return pos, vec

In [5]:
# get the supported device
device = get_device()
print(device)

# Data and constants
plummer_data_1a = pd.read_csv('plummer_regular.csv')
r = plummer_data_1a.loc[:,['r_x', 'r_y', 'r_z']]
r = r.to_numpy()
v = plummer_data_1a.loc[:,['v_x', 'v_y', 'v_z']]
v = v.to_numpy()
mass = np.full((10000,1), 2)

# convert from np.ndarray to torch.tensor
r = torch.from_numpy(r)
v = torch.from_numpy(v)
mass = torch.from_numpy(mass)

# simulation time
T = 1
dt = 1
step = T/dt
npoint = 10000

# result container
pos_out = torch.tensor((step, npoint, 3)).to(device)
vec_out = torch.tensor((step, npoint, 3)).to(device)
print('shape: ', pos_out.size())

#######################
# gpu computing testing
x = torch.rand(5, 3)
print(x)
print(type(r))
print(type(x))
print("Tensor device:", x.device)

print("CUDA GPU:", torch.cuda.is_available())
if torch.cuda.is_available():
   x = x.to("cuda:0")
   # or x=x.to("cuda")
print(x)

# now check the tensor device
print("Tensor device:", x.device)

cpu
shape:  torch.Size([3])
tensor([[0.3082, 0.0027, 0.6662],
        [0.6923, 0.7190, 0.9847],
        [0.0901, 0.1171, 0.8238],
        [0.9609, 0.4824, 0.8861],
        [0.6323, 0.0317, 0.3583]])
<class 'torch.Tensor'>
<class 'torch.Tensor'>
Tensor device: cpu
CUDA GPU: False
tensor([[0.3082, 0.0027, 0.6662],
        [0.6923, 0.7190, 0.9847],
        [0.0901, 0.1171, 0.8238],
        [0.9609, 0.4824, 0.8861],
        [0.6323, 0.0317, 0.3583]])
Tensor device: cpu


In [11]:
# x = np.array([3,27,5.3]).reshape(3,1)
# print(x.T-x)

In [ ]:
for i in range(int(step)):
    r, v = leapfrog(r, v, mass, G = 1, softening = 5, dt = dt)
    pos_out.append(r)
    vec_out.append(v)

pos_out = np.array(pos_out)
vec_out = np.array(vec_out)